# Los Angeles MIDI MetaBase (LAMB) Explorer and Tutorial Notebook (ver. 1.0)

***

## Kilo-Scale MIDI MIR Meta Database

## https://github.com/asigalov61/Los-Angeles-MIDI-Metabase

***

### Project Los Angeles

### Tegridy Code 2021

***

# Setup Environment

In [ ]:
# Install tegridy-tools

!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
# Import all needed modules

os.chdir('/content/')
print('Loading needed modules. Please wait...')

import sys
import os
import copy
import secrets
import random
import difflib
import json
from pathlib import Path
import pickle

os.chdir('/content/')

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

if not os.path.exists('/content/Output'):
    os.makedirs('/content/Output')

os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDI
import TMIDIX
import MIDI
import HaystackSearch
import tqdm

os.chdir('/content/')
print('Loading complete. Enjoy! :)')

# Download LAKH MIDI Dataset

In [ ]:
# Download and untar LAKH MIDI Dataset

%cd /content/
!wget http://hog.ee.columbia.edu/craffel/lmd/lmd_full.tar.gz
!tar -xvf lmd_full.tar.gz
%cd /content/

In [ ]:
# Create/save list of dataset files and count all of them

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')
%cd /content/
dataset_addr = "/content/lmd_full"
os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)
filez_l = len(filez)
print(len(filez))

TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/filez.pickle')

# LAMB Metabase

In [ ]:
# Download and unzip Los Angeles MIDI Metabase (LAMB)

%cd /content/

print('=' * 70)
print('Downloading pre-trained dataset-model...Please wait...')
print('=' * 70)

!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.001'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.002'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.003'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.004'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.005'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.006'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.007'
!wget 'https://github.com/asigalov61/Los-Angeles-MIDI-Metabase/raw/main/Metabase/Los-Angeles-MIDI-Metabase-210000.zip.008'

!cat Los-Angeles-MIDI-Metabase-210000.zip* > Los-Angeles-MIDI-Metabase-210000.zip
print('=' * 70)

!unzip -j Los-Angeles-MIDI-Metabase-210000.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /content/

In [ ]:
# Load the LAMB pickle file

LAMB = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/Los-Angeles-MIDI-Metabase-210000')

print('Done! :)')

# LAMB use example code/tutorial

In [ ]:
# Now we are going to have some fun... :)

# In the following excercise we are going to use LAMB to classify any source MIDI. 
# We are going to make a simple, yet powerful MIDI classifier (80%-90% accuracy)

# Let's begin...

# Load some MIDI file (absolutely any will do)
song = TMIDIX.Optimus_MIDI_TXT_Processor('/content/tegridy-tools/tegridy-tools/seed-lyrics.mid', 
                                         MIDI_channel=-1, 
                                         MIDI_patch=range(0, 127))

#  Compute some basic stats about it...
song_len = len(song[2])
song_start_times_avg = int(sum([y[1] for y in song[2]]) / song_len)
song_durations_avg = int(sum([y[2] for y in song[2]]) / song_len)
song_channels_avg = int(sum([y[3] for y in song[2]]) / song_len)
song_pitches_avg = int(sum([y[4] for y in song[2]]) / song_len)
song_velocities_avg = int(sum([y[5] for y in song[2]]) / song_len)

# ...and create a matching MIDI signature to compare against the dataset
Source_MIDI_Signature = [song_len, song_start_times_avg, 
                  song_durations_avg, song_channels_avg, 
                  song_pitches_avg, 
                  song_velocities_avg]

# Check the resuilts. They should have similar structure 
# There should be 5 entries if you did everything right
print('Source MIDI Signature:', Source_MIDI_Signature)

In [ ]:
# Here is a simple MIDI classifier/code...

sigs_match_ratings = []
sig_match_rating2 = []

for i in tqdm.tqdm(range(1, len(LAMB[2][1]))):
  # Computing signatures for each dataset file...
  Dataset_MIDI_Signature = [int(LAMB[2][1][i][0]),
                            int(LAMB[2][1][i][1] / (LAMB[2][1][i][0]+1)),
                            int(LAMB[2][1][i][2] / (LAMB[2][1][i][0]+1)),
                            int(LAMB[2][1][i][3] / (LAMB[2][1][i][0]+1)),
                            int(LAMB[2][1][i][4] / (LAMB[2][1][i][0]+1)),
                            int(LAMB[2][1][i][5] / (LAMB[2][1][i][0]+1))]
  # Computing ratings
  sig_match_rating = difflib.SequenceMatcher(None, ' '.join(map(str, Source_MIDI_Signature[2:])), ' '.join(map(str, Dataset_MIDI_Signature[2:]))).ratio()
  sigs_match_ratings.append(sig_match_rating)

# Computing some info
FNAME = LAMB[2][1][sigs_match_ratings.index(max(sigs_match_ratings))+1]
INFO = LAMB[2][0][sigs_match_ratings.index(max(sigs_match_ratings))+1][1:3]

# Info printout...
print(chr(10))
print(FNAME)
print(INFO[0])
print(INFO[1][0])
print(INFO[1][1])
print(sigs_match_ratings.index(max(sigs_match_ratings))+1)
print(max(sigs_match_ratings))

In [ ]:
# Simple code to search LAKH MIDI Dataset directory for MIDI matches

# If nothing found, MIDI is most likely NOT from LAKH MIDI Dataset

path = ''
print('Searching LAKH for:', INFO[0])
from pathlib import Path
for path in Path('/content/lmd_full').rglob(INFO[0]):
    print('File found!!!')
    print('The file below is the best possible match with the source MIDI:')
    print(path)

if len(str(path)) == 0:
  print('Could not locate the file. Check the code and the dataset.')

# Plot and Play your MIDI here

In [ ]:
# stuff for listening and plotting the results

!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!pip install pretty_midi

In [ ]:
# Plot and Play your MIDI file here...

# Full path to any MIDI file...
MIDI_FILE_TO_PLAY = '/content/tegridy-tools/tegridy-tools/seed3.mid'

# Code...
from midi2audio import FluidSynth
from IPython.display import display, Javascript, HTML, Audio
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
os.chdir('/content/')
print('Synthesizing the last output MIDI... ')
fname = str(MIDI_FILE_TO_PLAY)

fn = os.path.basename(MIDI_FILE_TO_PLAY)
fn1 = fn.split('.')[0]

print('Plotting the composition. Please wait...')

pm = pretty_midi.PrettyMIDI(fname)

# Retrieve piano roll of the MIDI file
piano_roll = pm.get_piano_roll()

plt.figure(figsize=(14, 5))
librosa.display.specshow(piano_roll, x_axis='time', y_axis='cqt_note', fmin=1, hop_length=160, sr=16000, cmap=plt.cm.hot)
plt.title(fn1)

FluidSynth("/usr/share/sounds/sf2/FluidR3_GM.sf2", 16000).midi_to_audio(str(fname), str('/content/composition' + '.wav'))
Audio(str('/content/composition' + '.wav'), rate=16000)